# 51. RNN의 기초 원리를 배워봅시다.

* 서울의 주간평균기온 데이터를 이용하여 RNN 모델을 구현해 봅시다.

## 1.환경 준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, SimpleRNN
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.random import set_seed

### (2) 필요한 함수 생성

* 학습곡선 함수

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

* 데이터 2 --> 3차원 변환

In [ ]:
# 시계열 데이터 전처리 2차원 --> 3차원으로 변환
def temporalize(x, y, timesteps):
    nfeature = x.shape[1]
    output_x = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        t = []
        for j in range(timesteps):
            t.append(x[[(i + j)], :])
        output_x.append(t)
        output_y.append(y[i + timesteps - 1])
    return np.array(output_x).reshape(-1,timesteps, nfeature), np.array(output_y)

### (3) Data Loading

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/temperature.csv')
data.head(10)

## 2.데이터 이해

온도의 흐름(trend)을 살펴봅시다.

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data.AvgTemp)
plt.grid()
plt.show()

##3.데이터 준비

### (1) y 만들기

In [ ]:
data['y'] = data['AvgTemp'].shift(-1)
data.dropna(axis = 0, inplace = True)
data.head()

### (2) x, y 분리

In [ ]:
x = data.loc[:, ['AvgTemp']]
y = data.loc[:,'y']

### (3) 스케일링

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [ ]:
x.shape, y.shape

### (4) 3차원 구조 만들기

In [ ]:
x2, y2 = temporalize(x, y, 4)
x2.shape, y2.shape

In [ ]:
x2[:2]

In [ ]:
 y[:7], y2[:2]

### (5) 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x2, y2, test_size= 53, shuffle = False)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_val.shape, y_val.shape

## 4.RNN : 무작정 모델링

### (1) 입력 구조(shape)
* 분석 단위 : 2차원 ( timesteps, nfeatures)

In [ ]:
x_train.shape

In [ ]:
timestep = x_train.shape[1]
nfeatures = x_train.shape[2]

### (2) 모델 구조 설계
* 다음 구조를 그대로 설계해 봅시다.
    * SimpleRNN(8, input_shape = (timestep, nfeatures))
    * Dense(1)

In [ ]:
clear_session()

model = Sequential([ 
    SimpleRNN(8, input_shape = (timestep, nfeatures)),
    Dense(1)
])

model.summary()

### (3) 컴파일 및 학습
* 다음 조건으로 학습해 봅시다.
    * epochs = 100
    * learning_rate = 0.01

In [ ]:
model.compile(optimizer= Adam(lr = 0.01), loss='mse')
history = model.fit(x_train, y_train, epochs = 100, batch_size = 64,
                    validation_split=.2).history

In [ ]:
# 학습 곡선을 그려봅시다.
dl_history_plot(history)

### (4) 예측 및 평가

In [ ]:
# 예측
pred = model.predict(x_val)

In [ ]:
# 평가
print(mean_squared_error(y_val, pred, squared=False))
print(mean_absolute_error(y_val, pred))
print(mean_absolute_percentage_error(y_val, pred))

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(y_val, label = 'actual')
plt.plot(pred, label = 'predicted')
plt.legend()
plt.grid()
plt.show()

## 5.RNN 모델링②

### (1) 입력 구조(shape)
* 분석 단위 : 2차원 ( timesteps, nfeatures)

In [ ]:
timestep = x_train.shape[1]
nfeatures = x_train.shape[2]

### (2) 모델 구조 설계
* 이제 RNN 레이어를 추가해 봅시다.
    * SimpleRNN(8, input_shape = (timestep, nfeatures), return_sequences = True)
    * SimpleRNN(4)
    * Dense(1)

### (3) 컴파일 및 학습
* 학습을 위한 조건은 직접 조정해 봅시다.

### (4) 예측 및 평가

## 6.RNN 모델링③
* 다음 항목을 조정하며 모델링을 수행해 봅시다.
    * timestep
    * 레이어와 노드수
    * learning_rate, epochs


### (1) 데이터 준비

* timestep 

In [ ]:
timestep = 
x2, y2 = temporalize(x, y, timestep)
x2.shape, y2.shape

* 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x2, y2, test_size= 53, shuffle = False)

### (2) 입력 구조(shape)
* 분석 단위 : 2차원 ( timesteps, nfeatures)

In [ ]:
timestep = x_train.shape[1]
nfeatures = x_train.shape[2]

### (3) 모델 구조 설계


### (4) 컴파일 및 학습

### (5) 예측 및 평가